# ChatGPT Sentiment Analysis of Facebook group comments

In [2]:

import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
from silvhua import *
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [4]:
from wrangling import filter_df_all_conditions, filter_df_any_condition

# *End of Page*